<a href="https://colab.research.google.com/github/yeesem/Deep_Learning/blob/main/Introduce_to_Keras_Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping,LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

In [3]:
# Download and prepare the horses or human dataset
path = './tensorflow_datasets'
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples,validation_examples,test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling tensorflow_datasets/horses_or_humans/3.0.0.incompleteIANZ48/horses_or_humans-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling tensorflow_datasets/horses_or_humans/3.0.0.incompleteIANZ48/horses_or_humans-test.tfrecord*...:   0%…

Dataset horses_or_humans downloaded and prepared to ./tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [4]:
SIZE = 150
IMAGE_SIZE = (SIZE,SIZE)

In [5]:
def format_image(image,label):
  image = tf.image.resize(image,IMAGE_SIZE)/255.0
  return image,label

In [6]:
BATCH_SIZE = 32

In [7]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [8]:
for image_batch,label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [19]:
def build_model(dense_units , input_shape = IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16,(3,3),activation = "relu",input_shape = input_shape),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units,activation = 'relu'),
      tf.keras.layers.Dense(2,activation = 'softmax')
      ])
  return model

# TensorBoard

In [17]:
!rm -rf logs

In [20]:
model = build_model(dense_units = 256)
model.compile(
    optimizer = 'sgd',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

logdir = os.path.join('logs',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(
    train_batches,
    epochs = 10,
    validation_data = validation_batches,
    callbacks = [tensorboard_callback]
)

Epoch 1/10
26/26 [==============================] - 25s 872ms/step - loss: 0.6767 - accuracy: 0.5633 - val_loss: 0.6655 - val_accuracy: 0.7220
Epoch 2/10
26/26 [==============================] - 19s 742ms/step - loss: 0.6407 - accuracy: 0.7178 - val_loss: 0.6228 - val_accuracy: 0.7756
Epoch 3/10
26/26 [==============================] - 21s 811ms/step - loss: 0.5811 - accuracy: 0.7664 - val_loss: 0.5405 - val_accuracy: 0.8049
Epoch 4/10
26/26 [==============================] - 27s 1s/step - loss: 0.5176 - accuracy: 0.7737 - val_loss: 0.4969 - val_accuracy: 0.8098
Epoch 5/10
26/26 [==============================] - 29s 1s/step - loss: 0.4480 - accuracy: 0.8066 - val_loss: 0.4034 - val_accuracy: 0.8341
Epoch 6/10
26/26 [==============================] - 19s 746ms/step - loss: 0.3910 - accuracy: 0.8406 - val_loss: 0.3332 - val_accuracy: 0.8585
Epoch 7/10
26/26 [==============================] - 18s 708ms/step - loss: 0.3599 - accuracy: 0.8443 - val_loss: 0.3401 - val_accuracy: 0.8829
Epoch

# Model Checkpoint

In [21]:
model = build_model(dense_units = 256)
model.compile(
    optimizer = 'sgd',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches,
    verbose = 2,
    callbacks = [
        ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1)
    ]
)

Epoch 1/5

Epoch 1: saving model to weights.01-0.67.h5
26/26 - 30s - loss: 0.6722 - accuracy: 0.5985 - val_loss: 0.6733 - val_accuracy: 0.5122 - 30s/epoch - 1s/step
Epoch 2/5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: saving model to weights.02-0.62.h5
26/26 - 21s - loss: 0.6418 - accuracy: 0.6837 - val_loss: 0.6225 - val_accuracy: 0.8098 - 21s/epoch - 811ms/step
Epoch 3/5

Epoch 3: saving model to weights.03-0.62.h5
26/26 - 27s - loss: 0.5965 - accuracy: 0.7360 - val_loss: 0.6161 - val_accuracy: 0.6049 - 27s/epoch - 1s/step
Epoch 4/5

Epoch 4: saving model to weights.04-0.52.h5
26/26 - 18s - loss: 0.5432 - accuracy: 0.7372 - val_loss: 0.5168 - val_accuracy: 0.8049 - 18s/epoch - 685ms/step
Epoch 5/5

Epoch 5: saving model to weights.05-0.44.h5
26/26 - 19s - loss: 0.4815 - accuracy: 0.7981 - val_loss: 0.4426 - val_accuracy: 0.7951 - 19s/epoch - 726ms/step


# Early Stopping

In [23]:
'''
min_delta: This parameter specifies the minimum change in the monitored metric
          to qualify as an improvement. If the change in the monitored metric is less
          than min_delta, it's not considered as an improvement. Early stopping will only
          trigger if the improvement is greater than min_delta. This helps prevent stopping
          training prematurely due to minor fluctuations in the metric.
baseline: This parameter sets the threshold value for the monitored metric. If the monitored
          metric doesn't improve by at least min_delta from the baseline value, early stopping is
          triggered. It's essentially the initial value from which improvements are measured. If
          the monitored metric doesn't exceed this baseline plus min_delta, training will stop.
'''

model = build_model(dense_units = 256)
model.compile(
    optimizer = 'sgd',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(
    train_batches,
    epochs = 50,
    validation_data = validation_batches,
    verbose = 2,
    callbacks = [
        EarlyStopping(
            patience = 3,
            min_delta = 0.05,
            baseline = 0.8,
            mode = 'min',
            monitor = 'val_loss',
            restore_best_weights = True,
            verbose = 1
        )
    ]
)

Epoch 1/50
26/26 - 22s - loss: 0.6815 - accuracy: 0.5487 - val_loss: 0.6886 - val_accuracy: 0.4341 - 22s/epoch - 848ms/step
Epoch 2/50
26/26 - 19s - loss: 0.6561 - accuracy: 0.6022 - val_loss: 0.6473 - val_accuracy: 0.6927 - 19s/epoch - 723ms/step
Epoch 3/50
26/26 - 17s - loss: 0.6246 - accuracy: 0.7482 - val_loss: 0.6122 - val_accuracy: 0.7122 - 17s/epoch - 658ms/step
Epoch 4/50
26/26 - 18s - loss: 0.5655 - accuracy: 0.7932 - val_loss: 0.5298 - val_accuracy: 0.8683 - 18s/epoch - 692ms/step
Epoch 5/50
26/26 - 18s - loss: 0.4997 - accuracy: 0.8151 - val_loss: 0.4574 - val_accuracy: 0.8049 - 18s/epoch - 674ms/step
Epoch 6/50
26/26 - 19s - loss: 0.4155 - accuracy: 0.8601 - val_loss: 0.3632 - val_accuracy: 0.9122 - 19s/epoch - 727ms/step
Epoch 7/50
26/26 - 21s - loss: 0.3324 - accuracy: 0.8905 - val_loss: 0.2688 - val_accuracy: 0.9317 - 21s/epoch - 824ms/step
Epoch 8/50
26/26 - 20s - loss: 0.2550 - accuracy: 0.9258 - val_loss: 0.3307 - val_accuracy: 0.8829 - 20s/epoch - 780ms/step
Epoch 9/

# CSV Logger

In [24]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

csv_file = 'training.csv'

model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches,
    callbacks = [
        CSVLogger(csv_file)
    ]
)

Epoch 1/5
26/26 [==============================] - 30s 1s/step - loss: 0.6857 - accuracy: 0.5852 - val_loss: 0.6514 - val_accuracy: 0.6244
Epoch 2/5
26/26 [==============================] - 20s 757ms/step - loss: 0.6288 - accuracy: 0.6533 - val_loss: 0.6178 - val_accuracy: 0.6488
Epoch 3/5
26/26 [==============================] - 19s 744ms/step - loss: 0.5729 - accuracy: 0.6922 - val_loss: 0.5296 - val_accuracy: 0.7854
Epoch 4/5
26/26 [==============================] - 19s 747ms/step - loss: 0.5316 - accuracy: 0.7518 - val_loss: 0.6036 - val_accuracy: 0.6195
Epoch 5/5
26/26 [==============================] - 18s 690ms/step - loss: 0.4726 - accuracy: 0.8187 - val_loss: 0.4828 - val_accuracy: 0.7220


# Learning Rate Scheduler

In [29]:
'''
drop: This parameter specifies the factor by which the learning rate is reduced.
      After every epochs_drop epochs, the learning rate will be multiplied by this factor.
      For example, if drop is set to 0.5, it means the learning rate will be halved every
      epochs_drop epochs.

epochs_drop: This parameter determines how often (in terms of epochs) the learning rate
            will be updated. After every epochs_drop epochs, the learning rate will be adjusted
            according to the specified drop factor.
'''

model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

def step_decay(epoch):
  initial_lr = 0.01
  drop = 0.5
  epochs_drop = 1
  lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lr

model.fit(train_batches,
          epochs=5,
          validation_data=validation_batches,
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 1: LearningRateScheduler setting learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 22s 830ms/step - loss: 0.6716 - accuracy: 0.5779 - val_loss: 0.6813 - val_accuracy: 0.4829 - lr: 0.0050

Epoch 2: LearningRateScheduler setting learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 22s 858ms/step - loss: 0.6326 - accuracy: 0.6667 - val_loss: 0.6362 - val_accuracy: 0.7268 - lr: 0.0025

Epoch 3: LearningRateScheduler setting learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 22s 872ms/step - loss: 0.6172 - accuracy: 0.7129 - val_loss: 0.6404 - val_accuracy: 0.6098 - lr: 0.0012

Epoch 4: LearningRateScheduler setting learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 18s 677ms/step - loss: 0.6080 - accuracy: 0.7226 - val_loss: 0.6267 - val_accuracy: 0.6976 - lr: 6.2500e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0003125.
Epoch 5/5
26/26 [=============================

# ReduceLROnPlateau
Reduce learning rate when a metric has stopped improving.<br>
Adjust the learning rate dynamically based on the validation loss.

In [33]:
'''
min_lr: This parameter specifies the minimum value that the learning rate can reach.
        If, during training, the learning rate falls below this value, it will not be further
        reduced. This helps prevent the learning rate from becoming too small, which could
        hinder convergence.

factor: This parameter determines the factor by which the learning rate will be reduced.
        If the validation loss plateaus (stops improving), the learning rate will be updated by
        multiplying it by this factor. For example, if factor is set to 0.2, it means that the
        learning rate will be reduced to 20% of its current value whenever the validation
        loss plateaus.
'''

model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(train_batches,
          epochs=50,
          validation_data=validation_batches,
          callbacks=[ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 21s 794ms/step - loss: 0.6700 - accuracy: 0.6107 - val_loss: 0.6316 - val_accuracy: 0.8049 - lr: 0.0100
Epoch 2/50
26/26 [==============================] - 21s 796ms/step - loss: 0.6203 - accuracy: 0.6703 - val_loss: 0.5902 - val_accuracy: 0.7805 - lr: 0.0100
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 0.5625 - accuracy: 0.7360
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 18s 711ms/step - loss: 0.5625 - accuracy: 0.7360 - val_loss: 0.7039 - val_accuracy: 0.4878 - lr: 0.0100
Epoch 4/50
26/26 [==============================] - 18s 704ms/step - loss: 0.5137 - accuracy: 0.8066 - val_loss: 0.4972 - val_accuracy: 0.8976 - lr: 0.0020
Epoch 5/50
26/26 [==============================] - 17s 655ms/step - loss: 0.4887 - accuracy: 0.8491 - val_loss: 0.4890 - val_accuracy: 0.8488 - lr: 0.0020
Epoch 6/50
26/26 [==============================] - 20s 777ms